In [2]:
import sys
import os
os.chdir("../../")
import slim

In [3]:
"""
A script that downloads and parses Salmon Scotland mortality reports.
"""

from bs4 import BeautifulSoup
from requests import get
import os
from pathlib import Path
import tabula
import numpy as np

WEBSITE = "https://www.salmonscotland.co.uk"
REPORT_URL = f"{WEBSITE}/reports/monthly-mortality-rate-%s-%d"


def download(month: str, year: int):
    report_out_folder = Path("output/reports/")
    if year == 2018:
        filename = report_out_folder / f"SS-{year}.pdf"
    else:    
        filename = report_out_folder / f"SS-{month}-{year}.pdf"
    os.makedirs(str(report_out_folder), exist_ok=True)
    
    if not filename.exists():
        if year == 2018:
            # All monthly data were merged in a single pdf apparently
            url = "https://www.salmonscotland.co.uk/reports/scottish-salmon-survival-rates-2018"
        else:
            url = REPORT_URL % (month, year)
        parse_page = get(url).content
        parser = BeautifulSoup(parse_page, "html.parser")
        div = parser.find("div", class_="download-link")
        a = div.find("a")
        download_link = WEBSITE + a["href"]

        report_out_folder = Path("output/reports/")
        if year == 2018:
            report_out_folder / f"SS-{year}.pdf"
        else:
            filename = report_out_folder / f"SS-{month}-{year}.pdf"
        os.makedirs(str(report_out_folder), exist_ok=True)
        downloaded_pdf = get(download_link)

        with filename.open("wb") as f:
            f.write(downloaded_pdf.content)

    return filename

In [4]:
import pandas as pd
import traceback

column_names = ["company", "site", "mortality", "note", "cumulative_mortality"]

def get_pdf_page(pdf, page, month, year):
    # parsing of multiple pages breaks
    candidate = tabula.read_pdf(pdf, pages=page, pandas_options={"header": "none"})
    
    if len(candidate) == 0:
        return pd.DataFrame({column: {} for column in column_names})
    df = candidate[0]
    if page == 2:
        df = df.iloc[3:].reset_index(drop=True)
    
    if len(df.columns) == 4:
        cumulative_mort = df[3].copy()
        df[3] = np.nan
        df[4] = cumulative_mort
    elif len(df.columns) == 3:
        df[3] = np.nan
        df[4] = np.nan
        
    df = df.set_axis(column_names, axis=1)
    

    df["month"] = month
    df["year"] = year
    df["year"] = df["year"].astype(np.int64)
    return df

def get_pdf_pages(pdf, month, year):
    if (month == "December" and year == 2020):
        range_ = range(1, 6)
        
    elif year == 2018:
        # merged pdfs, regular except for september and october
        month_no = datetime.datetime.strptime(month, "%B").month
        if month_no not in [9, 10, 12]:
            range_ = range(2+8*(month_no-1), 2+8*(month_no-1)+6)
        elif month == 9:
            range_ = range(66, 72)
        elif month == 10:
            range_ = range(73, 79)
        else:
            range_ = range(88, 95)
            
    elif year == 2021:
        if month in ["January", "March"]:
            range_ = range(1, 10)
        elif month in ["February"]:
            range_ = range(1, 9)
        elif month in ["August"]:
            range_ = range(3, 11)
        else:
            range_ = range(3, 10)
    else:
        range_ = range(2, 8)
    
    dfs = []
    for page in range_:
        try:
            dfs.append(get_pdf_page(pdf, page, month, year))
        except:
            traceback.print_exc()
        
    return pd.concat(dfs, ignore_index=True, sort=False)

In [5]:
import datetime


def extract_year_report(year):
    def parse_and_extract(month):
        print(f"Parsing {year}-{month}")
        month_label = datetime.date(year, month, 1).strftime('%B')
        try:
            location = download(month_label, year)
            return get_pdf_pages(location, month_label, year)
        except:
            print(f"Unexpected error when fetching {year}-{month}")
            traceback.print_exc()
            
    dfs = [parse_and_extract(month) for month in range(1, 13)]
        
    df = pd.concat(dfs, ignore_index=True)
    df[~df["company"].isnull()].reset_index(drop=True)
    df["year"] = df["year"].apply(np.uint64)
    
    return df

def collate_years(range_):
    return pd.concat((extract_year_report(y) for y in range_), ignore_index=True, sort=False)

In [6]:
# This is going to take some time

REPORT_LOCATION = "output/reports/SS-full-2018-2022.csv"
if not Path(REPORT_LOCATION).exists():
    df = collate_years(range(2018, 2022))
    df = df.replace({"Ardgadden": "Ardgaddan", "Ardcastle Bay": "Ardcastle"})
    df.to_csv(REPORT_LOCATION)
else:
    df = pd.read_csv(REPORT_LOCATION)

df

,Unnamed: 0,company,site,mortality,note,cumulative_mortality,month,year
0,0,Cooke Aquaculture (Scotland),Balta Isle,6.0,IPN,In production,January,2018
1,1,Cooke Aquaculture (Scotland),Bastaness,Fallow,NaN,Fallow,January,2018
2,2,Cooke Aquaculture (Scotland),Bay of Cleat (North),0.7,NaN,In production,January,2018
3,3,Cooke Aquaculture (Scotland),Bay of Cleat (South),Farm fallowed in Jan.,NaN,16.0,January,2018
4,4,Cooke Aquaculture (Scotland),Bay of Ham,Fallow,NaN,Fallow,January,2018
...,...,...,...,...,...,...,...,...
9320,9320,Scottish Sea Farms Ltd,Wyre,0.0,NaN,In production,December,2021
9321,9321,Wester Ross Fisheries Ltd,Ardessie A,0.1,NaN,In production,December,2021
9322,9322,Wester Ross Fisheries Ltd,Ardessie B,0.3,NaN,In production,December,2021
9323,9323,Wester Ross Fisheries Ltd,Ardmair,0.7,NaN,In production,December,2021


## Marine Scotland parsing

In [7]:
import json
import re

MS_REPORT_JSON = "https://data.marine.gov.scot/api/3/action/package_show?id=55aa8a12-135e-463e-802b-fb661fa02b73&page=0"
REPORT_JSON_LOCATION = Path("output/reports")

def get_lice_counts_json(year):
    report_json_location = REPORT_JSON_LOCATION / f"MS_{year}.csv"
    if not report_json_location.exists():
        parsed = json.loads(get(MS_REPORT_JSON).content)
        titles_urls = [(res["name"], res["url"]) for res in parsed["result"][0]["resources"]]
        titles, urls = zip(*titles_urls)

        ranges = [range(*tuple(map(int, 
                      re.findall(r"(\d+)-(\d+)", s)[0])
                             )) for s in titles]
        url = next(url for idx, url in enumerate(urls) if year in ranges[idx])
        with report_json_location.open("wb") as f:
            response = get(url, headers={'Content-type': 'application/json'})
            f.write(response.content)

    return report_json_location

def get_lice_counts(year):
    return pd.read_csv(str(get_lice_counts_json(year)))

In [8]:
lice_counts_2019 = get_lice_counts(2019)

In [9]:
lice_counts_2019

,Site ID,Site Name,Year,Month,Lice Count,Comments
0,BALT1,Balta Island,2018,January,0.03,NaN
1,BALT1,Balta Island,2018,February,0.07,NaN
2,BALT1,Balta Island,2018,March,0.15,NaN
3,BALT1,Balta Island,2018,April,0.1,NaN
4,BALT1,Balta Island,2018,May,0.06,NaN
...,...,...,...,...,...,...
8274,BRO1,"Corry, Loch Broom",2020,December,0,NaN
8275,WHA2,North Voe,2020,December,F,NaN
8276,MCLN1,MacLeans Nose,2020,December,0,NaN
8277,KIL1,Petersport South (Kilerivagh),2020,December,F,NaN


In [10]:
lice_counts_2019["Site Name"]

0                        Balta Island
1                        Balta Island
2                        Balta Island
3                        Balta Island
4                        Balta Island
                    ...              
8274                Corry, Loch Broom
8275                        North Voe
8276                    MacLeans Nose
8277    Petersport South (Kilerivagh)
8278                     Puldrite Bay
Name: Site Name, Length: 8279, dtype: object

## Extract Fyne and Lynnhe

In [11]:
from slim.simulation.config import Config

config = Config("config_data/config.json", "config_data/Fyne_complete")
farm_names = [farm.name for farm in config.farms]
farm_names

['Tarbert South',
 'Rubha Stillaig',
 'Glenan Bay',
 'Meall Mhor',
 'Gob a Bharra',
 'Strondoir Bay',
 'Ardgaddan',
 'Ardcastle',
 'Quarry Point']

In [12]:
df_fyne = df[df["site"].isin(farm_names)]
df_fyne

,Unnamed: 0,company,site,mortality,note,cumulative_mortality,month,year
113,113,Scottish Salmon Company Ltd,Ardcastle,0.4,NaN,In production,January,2018
114,114,Scottish Salmon Company Ltd,Ardgaddan,0.6,NaN,In production,January,2018
122,122,Scottish Salmon Company Ltd,Glenan Bay,0.4,NaN,In production,January,2018
123,123,Scottish Salmon Company Ltd,Gob a Bharra,0.4,NaN,In production,January,2018
134,134,Scottish Salmon Company Ltd,Meall Mhor,0.2,NaN,In production,January,2018
...,...,...,...,...,...,...,...,...
9258,9258,Scottish Salmon Company Ltd,Meall Mhor,1.2,NaN,In production,December,2021
9262,9262,Scottish Salmon Company Ltd,Quarry Point,0.5,NaN,In production,December,2021
9264,9264,Scottish Salmon Company Ltd,Rubha Stillaig,Fallow,NaN,Fallow,December,2021
9270,9270,Scottish Salmon Company Ltd,Strondoir Bay,1.0,NaN,In production,December,2021


In [13]:
fyne_sites_data = lice_counts_2019[lice_counts_2019["Site Name"].isin(farm_names)]
fyne_sites_data

,Site ID,Site Name,Year,Month,Lice Count,Comments
1499,FFMC43,Ardcastle,2018,January,0,NaN
1500,FFMC43,Ardcastle,2018,February,0,NaN
1501,FFMC43,Ardcastle,2018,March,0,NaN
1502,FFMC43,Ardcastle,2018,April,0,NaN
1503,FFMC43,Ardcastle,2018,May,0,NaN
...,...,...,...,...,...,...
8208,FFMC14,Meall Mhor,2020,December,0.28,NaN
8213,FFMC29,Quarry Point,2020,December,0.43,NaN
8215,FFMC66,Rubha Stillaig,2020,December,0.17,NaN
8220,FFMC70,Strondoir Bay,2020,December,0.34,NaN


In [14]:
lice_counts_2019["Site Name"].unique()

array(['Balta Island', 'West of Skeo Taing site (Balta Harbour)',
       'Bastaness', 'Basta Voe North West (Kirkabister)',
       'Bay of Cleat North', 'Bay of Cleat', 'Bay of Ham', 'Bay of Vady',
       'Bow of Hascosay', 'Bay of Meil', 'Point of Burkwell (Site 5)',
       'Burrastow', 'Carness Bay', 'Chalmers Hope', 'Cloudin',
       'Cava South', 'Djuba Wick', 'West Fara', 'Skelwick Skerry',
       'Flaeshins', 'Hogan (Site 1)', 'Kirk Noust', 'Lyrawa Bay',
       'Mid Taing', 'Ness of Copister', 'North Sandwick', 'Ouse Ness',
       'Pegal Bay', 'Quanterness (West Shargun Shoal)',
       'Stead of Aithness', 'Turness', 'Uyea Isle', 'Swarta Skerry',
       'Vee Taing', 'Vest Ness', 'Wick of Belmont North',
       'Wick of Vatsetter', 'Winna Ness', 'Setterness East (Bomlo)',
       'Bight of Foraness', 'Cole Deep', 'Collafirth Delting Site 3',
       'Corlarach', 'Papa, East Head of Scalloway', 'East of Langa',
       'East of Papa Little', 'Easter Score Holm',
       'Geo of Vallada

In [15]:
fyne_sites_data["Site Name"].unique()

array(['Ardcastle', 'Ardgaddan', 'Glenan Bay', 'Gob a Bharra',
       'Meall Mhor', 'Quarry Point', 'Rubha Stillaig', 'Strondoir Bay',
       'Tarbert South'], dtype=object)

In [16]:
# We are ready to join the two dataframes

fyne_data_joined = fyne_sites_data.set_index(["Year", "Month", "Site Name"])\
                    .join(df_fyne.set_index(["year", "month", "site"]).rename_axis(["Year", "Month", "Site Name"]),
                          how="right")
fyne_data_joined

Site ID Lice Count Comments  Unnamed: 0  \
Year Month    Site Name                                                
2018 January  Ardcastle       FFMC43          0      NaN         113   
              Ardgaddan       FFMC47       0.01      NaN         114   
              Glenan Bay      FFMC10          0      NaN         122   
              Gob a Bharra    FFMC46          0      NaN         123   
              Meall Mhor      FFMC14          0      NaN         134   
...                              ...        ...      ...         ...   
2021 December Meall Mhor         NaN        NaN      NaN        9258   
              Quarry Point       NaN        NaN      NaN        9262   
              Rubha Stillaig     NaN        NaN      NaN        9264   
              Strondoir Bay      NaN        NaN      NaN        9270   
              Tarbert South      NaN        NaN      NaN        9273   

                                                  company mortality note  \
Year Month    Site Name                                                    
2018 January  Ardcastle       Scottish Salmon Company Ltd       0.4  NaN   
              Ardgaddan       Scottish Salmon Company Ltd       0.6  NaN   
              Glenan Bay      Scottish Salmon Company Ltd       0.4  NaN   
              Gob a Bharra    Scottish Salmon Company Ltd       0.4  NaN   
              Meall Mhor      Scottish Salmon Company Ltd       0.2  NaN   
...                                                   ...       ...  ...   
2021 December Meall Mhor      Scottish Salmon Company Ltd       1.2  NaN   
              Quarry Point    Scottish Salmon Company Ltd       0.5  NaN   
              Rubha Stillaig  Scottish Salmon Company Ltd    Fallow  NaN   
              Strondoir Bay   Scottish Salmon Company Ltd       1.0  NaN   
              Tarbert South   Scottish Salmon Company Ltd       1.6  NaN   

                             cumulative_mortality  
Year Month    Site Name                            
2018 January  Ardcastle             In production  
              Ardgaddan             In production  
              Glenan Bay            In production  
              Gob a Bharra          In production  
              Meall Mhor            In production  
...                                           ...  
2021 December Meall Mhor            In production  
              Quarry Point          In production  
              Rubha Stillaig               Fallow  
              Strondoir Bay         In production  
              Tarbert South         In production  

[403 rows x 8 columns]

In [17]:
# final clean-ups

def clean_up(df):
    def replace(mortality):
        if (x := re.match("(\d+(\.\d+)?)", mortality)):
            return float(x.group())
        return np.nan

    df_copy = df.copy()


    df_copy["mortality"] = df_copy["mortality"].fillna("").apply(replace)
    df_copy["lice_count"] = df_copy["Lice Count"].fillna("").apply(replace)
    df_copy = df_copy.rename({
        "Comments": "mortality_comment",
        "Site ID": "site_id",
        "note": "lice_note"
    }, axis=1)[["site_id", "lice_count", "mortality", "cumulative_mortality", "lice_note", "mortality_comment"]]

    return df_copy

fyne_data_df = clean_up(fyne_data_joined)


In [18]:
from datetime import datetime

def to_day(x):
    return datetime.strptime(f"{x['Year']} {x['Month']} 15", "%Y %B %d")


fyne_data_df = fyne_data_df.reset_index()
fyne_data_df["date"] = fyne_data_df.apply(to_day, axis=1)
fyne_data_df = fyne_data_df.drop(columns=["Year", "Month"])\
                            .set_index(["date", "Site Name"])\
                            .rename_axis(index={"Site Name": "site_name"})
fyne_data_df

site_id  lice_count  mortality cumulative_mortality  \
date       site_name                                                            
2018-01-15 Ardcastle       FFMC43        0.00        0.4        In production   
           Ardgaddan       FFMC47        0.01        0.6        In production   
           Glenan Bay      FFMC10        0.00        0.4        In production   
           Gob a Bharra    FFMC46        0.00        0.4        In production   
           Meall Mhor      FFMC14        0.00        0.2        In production   
...                           ...         ...        ...                  ...   
2021-12-15 Meall Mhor         NaN         NaN        1.2        In production   
           Quarry Point       NaN         NaN        0.5        In production   
           Rubha Stillaig     NaN         NaN        NaN               Fallow   
           Strondoir Bay      NaN         NaN        1.0        In production   
           Tarbert South      NaN         NaN        1.6        In production   

                          lice_note mortality_comment  
date       site_name                                   
2018-01-15 Ardcastle            NaN               NaN  
           Ardgaddan            NaN               NaN  
           Glenan Bay           NaN               NaN  
           Gob a Bharra         NaN               NaN  
           Meall Mhor           NaN               NaN  
...                             ...               ...  
2021-12-15 Meall Mhor           NaN               NaN  
           Quarry Point         NaN               NaN  
           Rubha Stillaig       NaN               NaN  
           Strondoir Bay        NaN               NaN  
           Tarbert South        NaN               NaN  

[403 rows x 6 columns]

## Extract the actions

The report above contains some information about treamtents being applied - except that it does not tell us which is being applied

In [19]:
fyne_data_df.to_csv("config_data/Fyne_complete/report.csv")

In [20]:
# to convert week day to 
from datetime import datetime
def first_monday(year):
    weekday = datetime(day=1, month=1, year=year).weekday()
    return datetime(day=1 + (7 - weekday) % 7, month=1, year=year)
    
first_monday(2018)

datetime.datetime(2018, 1, 1, 0, 0)

In [21]:
import pandas as pd
import csv

# I don't want to do excel->csv conversion here.
# Structure: site no, site name, business name, business no,
#    {count, mitigation, sg action}*52 times (53 in 2020)

import datetime as dt

def load_weekly(year):
    builder = {}
    with open(f"slim/surveys/ms_weekly_data_{year}.csv", "r") as f:
        # skip the first two lines
        for _ in range(2):
            f.readline()
        day = first_monday(year)
        data = list(csv.reader(f))
        builder["time"] = []
        builder["year"] = []
        builder["month"] = []
        builder["site_id"] = []
        builder["site_name"] = []
        builder["mitigation"] = []
        for line in data:
            site_no = line[0]
            site_name = line[1]
            step = 3
            if year == 2020:
                step = 4
            for i in range(4, 4 + step*52, step):
                time = day + dt.timedelta(days=(i-4) // step)
                week_data = line[i:i+3]
                mitigation = week_data[1]
                builder["time"].append(time)
                builder["year"].append(time.year)
                builder["month"].append(time.month)
                builder["site_id"].append(site_no)
                builder["site_name"].append(site_name)
                builder["mitigation"].append(mitigation)
    return pd.DataFrame(builder)

In [22]:
weekly_data_df = pd.concat([load_weekly(year) for year in range(2018, 2021)])

In [23]:
weekly_data_df

,time,year,month,site_id,site_name,mitigation
0,2018-01-01,2018,1,FS0242,Loch Odhairn/Gravir,
1,2018-01-02,2018,1,FS0242,Loch Odhairn/Gravir,Physical
2,2018-01-03,2018,1,FS0242,Loch Odhairn/Gravir,Physical
3,2018-01-04,2018,1,FS0242,Loch Odhairn/Gravir,Physical
4,2018-01-05,2018,1,FS0242,Loch Odhairn/Gravir,
...,...,...,...,...,...,...
3947,2020-02-22,2020,2,FS0407,Wick of Vatsetter,
3948,2020-02-23,2020,2,FS0407,Wick of Vatsetter,
3949,2020-02-24,2020,2,FS0407,Wick of Vatsetter,
3950,2020-02-25,2020,2,FS0407,Wick of Vatsetter,


In [24]:
x = weekly_data_df["mitigation"]

In [25]:
x.unique() # treatments

array(['', 'Physical', 'EoC', 'Harvesting', 'Bath', 'Cleanerfish',
       'Physical & Harvesting', 'Bath & Physical', 'In Feed',
       'Bio Reduction', 'Destocking', '3.47'], dtype=object)

In [26]:
fyne_data_df = pd.read_csv("config_data/Fyne_complete/report.csv")

In [27]:
weekly_data_df["site_name"].unique()

array(['Loch Odhairn/Gravir', 'Strome', 'Trilleachan Mor', 'Portree',
       'Inch Kenneth', 'Druimyeon Bay', 'Geasgill', 'Eughlam', 'Vuiabeag',
       'Kyles of Vuia', 'Vuia Mor', 'Vacasay', 'Taranaish',
       'Bagh Chlann Neill', 'Plocrapol', 'Reibinish', 'Scadabay',
       'Ardcastle Bay', 'Kenmore Loch Torridon', 'Outer Eport',
       'Furnace Quarry', 'Tarbert South', 'Fishnish (B)', 'Fishnish (A)',
       'Loch Spelve (B)', 'Slocka Ronas Voe', 'Vidlin North',
       'Bight of Bellister, Dury Voe', 'Invasion Bay', 'North Shore',
       'Loch Duich', 'Seaforth', "MacLean's Nose", 'Ornish Island',
       'Loch Snizort East', 'Swining Voe 3', 'Poseidon',
       'Setterness North (Bomlo)', 'Linga (Setterness)', 'Corlarach',
       'Gob na Hoe', 'Leinish', 'Score Holms', 'Setter Voe', 'North Papa',
       'Langa Isle (East)', 'Turness', 'Djubawick', 'Bastaness',
       'Wick of Vatsetter', 'Ness of Copister',
       'East of Holm Heogland (Burkwell)', 'Uyea Isle', 'Vee Taing',
       

In [28]:
# Not good! Almost none of loch fyne farms are there. We can resort to SC reports (although much coarser)!

notes = fyne_data_df["lice_note"].unique()

print(notes)

[nan 'Environmental' 'Treatment / Handling' 'Handling'
 'Environmental challenges' 'Environmental challenges/Gill'
 'Gill Health/Treatment losses' 'Gill Health/Handling' 'Gill Health'
 'Sea lice management / Viral disease' 'Handling / Sea lice management'
 'Sea lice management' 'Gill health related']


In [29]:
is_treating_note = [note for note in notes if isinstance(note, str) and "Sea lice" in note]
is_treating_note

['Sea lice management / Viral disease',
 'Handling / Sea lice management',
 'Sea lice management']

In [30]:
fyne_data_df

,date,site_name,site_id,lice_count,mortality,cumulative_mortality,lice_note,mortality_comment
0,2018-01-15,Ardcastle,FFMC43,0.00,0.4,In production,NaN,NaN
1,2018-01-15,Ardgaddan,FFMC47,0.01,0.6,In production,NaN,NaN
2,2018-01-15,Glenan Bay,FFMC10,0.00,0.4,In production,NaN,NaN
3,2018-01-15,Gob a Bharra,FFMC46,0.00,0.4,In production,NaN,NaN
4,2018-01-15,Meall Mhor,FFMC14,0.00,0.2,In production,NaN,NaN
...,...,...,...,...,...,...,...,...
398,2021-12-15,Meall Mhor,NaN,NaN,1.2,In production,NaN,NaN
399,2021-12-15,Quarry Point,NaN,NaN,0.5,In production,NaN,NaN
400,2021-12-15,Rubha Stillaig,NaN,NaN,NaN,Fallow,NaN,NaN
401,2021-12-15,Strondoir Bay,NaN,NaN,1.0,In production,NaN,NaN


In [31]:
%load_ext autoreload
%autoreload 2

In [32]:
from slim.types.treatments import Treatment
from slim.simulation.farm import Farm
from slim.simulation.simulator import Simulator, PilotedPolicy

from slim.types.policies import NO_ACTION, FALLOW
from slim.simulation.config import Config
from pathlib import Path

In [33]:
import pickle

cfg = Config("config_data/config.json", "config_data/Fyne_complete", save_rate=1, farms_per_process=1)
policy = PilotedPolicy(cfg, weekly_data_df, fyne_data_df)

with open("config_data/piloted_fyne.pkl", "wb") as f:
    pickle.dump(policy, f)

In [49]:
from slim.simulation.simulator import load_counts

report_df = load_counts(cfg)
print(report_df.columns)
report_df[report_df["site_name"]=="Ardcastle"]

[autoreload of ray.core.generated.runtime_env_common_pb2 failed: Traceback (most recent call last):
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/site-packages/ray/core/generated/runtime_env_common_pb2.py", line 16,

Index(['date', 'site_name', 'site_id', 'lice_count', 'mortality',
       'cumulative_mortality', 'lice_note', 'mortality_comment',
       'survived_fish'],
      dtype='object')


[autoreload of ray.client_builder failed: Traceback (most recent call last):
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/erolm_a/.conda/envs/slp/lib/python3.9/site-packages/ray/client_builder.py", line 28, in <module>
    class ClientContext(BaseConte

,date,site_name,site_id,lice_count,mortality,cumulative_mortality,lice_note,mortality_comment,survived_fish
0,2018-01-15,Ardcastle,FFMC43,0.00,0.40,In production,NaN,NaN,358560.0
9,2018-02-15,Ardcastle,FFMC43,0.00,0.20,In production,NaN,NaN,357843.0
18,2018-03-15,Ardcastle,FFMC43,0.00,0.60,In production,NaN,NaN,355696.0
27,2018-04-15,Ardcastle,FFMC43,0.00,0.60,In production,NaN,NaN,353562.0
36,2018-05-15,Ardcastle,FFMC43,0.00,0.20,In production,NaN,NaN,352855.0
45,2018-06-15,Ardcastle,FFMC43,0.04,0.10,In production,NaN,NaN,352502.0
54,2018-07-15,Ardcastle,FFMC43,0.04,0.20,In production,NaN,NaN,351797.0
63,2018-08-15,Ardcastle,FFMC43,0.26,0.30,In production,NaN,NaN,350741.0
72,2018-09-15,Ardcastle,FFMC43,0.52,2.20,In production,NaN,NaN,343025.0
76,2018-10-15,Ardcastle,FFMC43,1.98,2.20,In production,NaN,NaN,335478.0


In [52]:
data = report_df.groupby("site_name")[["survived_fish", "lice_count", "date"]].apply(lambda x: x.reset_index().to_dict()).to_dict()

data["Ardcastle"]["survived_fish"][0]

358560.0

In [53]:
ardcastle_farm = report_df[report_df["site_name"]=="Ardcastle"] 

x = ardcastle_farm["date"]\
    .searchsorted(dt.datetime(year=2021,month=12,day=15))

ardcastle_farm.iloc[x]

date                    2021-12-15 00:00:00
site_name                         Ardcastle
site_id                                 NaN
lice_count                              NaN
mortality                               0.5
cumulative_mortality          In production
lice_note                               NaN
mortality_comment                       NaN
survived_fish                      356409.0
Name: 394, dtype: object

In [51]:
sim = Simulator(Path("outputs/"), Path("Fyne_piloted"), cfg)
sim.policy = policy
sim.run_model()

2022-05-11 16:37:54,455	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265
  2%|█▍                                                      | 18/730 [00:04<03:09,  3.75it/s]

Stopping farms


In [16]:
import ray
ray.shutdown()

In [ ]:
import pandas as pd